In [133]:
import numpy as np
import csv
from itertools import permutations
import time

file = open('/home/johannes/Work/INF3490/oblig1/european_cities.csv', 'r+')
reader = csv.reader(file,delimiter=";")

distance = np.zeros((24,24))
cities = []

i = 0
for row in reader:
    #print row
    if i == 0:
        for j in range(len(row)):
            cities.append(row[j])
    else:
        for j in range(len(distance)):
            distance[i-1,j] = float(row[j])
    i += 1


In [188]:
# Exhaustive search

start_time = time.time()

N = 6
A = distance[0:N,0:N]

a = np.arange(N)

def dist(cities):
    tour = np.zeros(len(cities))
    for i in range(len(cities)-1):
        tour[i] = A[cities[i],cities[i+1]]
    tour[-1] = A[cities[-1], cities[0]]
    return np.sum(tour)
    
    
best_sequence = np.zeros(len(cities))
best = 100000000
for i in permutations(a):
    temp = dist(i)
    if temp < best:
        best = temp
        best_sequence = i
        
elapsed_time = time.time() - start_time
        
print best
print best_sequence
print ("Time elapsed is %f seconds" %(elapsed_time))


def index_to_city_name(city_names, city_indices):
    best_cities = []
    for i in range(len(city_indices)):
        best_cities.append(city_names[int(city_indices[i])])
    return best_cities

print index_to_city_name(cities, best_sequence)

5018.81
(0, 1, 4, 5, 2, 3)
Time elapsed is 0.006908 seconds
['Barcelona', 'Belgrade', 'Bucharest', 'Budapest', 'Berlin', 'Brussels']


In [185]:
# Hill climbing

def hillclimb(cities):
    init = cities
    np.random.shuffle(init)
    
    best = dist(init)
    count = 0
    best_sequence = init.copy()
    while count < 10:        
        number1 = np.random.randint(N)
        number2 = np.random.randint(N)
        
        while number1 == number2:
            number2 = np.random.randint(N)
        
        value1 = init[number1]
        value2 = init[number2]
        
        init[number1] = value2
        init[number2] = value1
        
        temp = dist(init) 
        
        if temp < best:
            best = temp
            best_sequence = init.copy()
            count = 0
        else:
            count += 1
        
    return best, best_sequence

best, best_sequence = hillclimb(a)
print best, best_sequence  
print index_to_city_name(cities, best_sequence)

# WTF happens with the zeros??
    

5783.0 [2 0 3 4 1 5]
['Berlin', 'Barcelona', 'Brussels', 'Bucharest', 'Belgrade', 'Budapest']


In [242]:
# Genetic algorithm

# Create population
def mk_population(size,cities):
    population = np.zeros((size,len(cities)))
    for i in range(size):
        np.random.shuffle(cities) 
        population[i] = cities
    return population

v = mk_population(6,a)
print v


# Create tournament group
def mk_tournament(population,size):
    
    tournament_size = population.shape[0]/size
    if (float(population.shape[0])/size).is_integer()!=True:
        print "Change population or tournament size!"
    print tournament_size
        
    tournaments = np.zeros((size,tournament_size))
    population_i = np.arange(population.shape[0])
    np.random.shuffle(population_i)
    
    print population_i
    count = 0
    for i in range(size):
        for j in range(tournament_size):
            tournaments[i,j] = population_i[count]
            count += 1
            
    
    return tournaments


w = mk_tournament(v,2)    
print w

# create parents
# takes in the tournaments matrix from mk_tournament, the population, and which tournament you want
def mk_parents(tournaments,population,tournament):
    
    actual_tournament = np.zeros((tournaments.shape[1],population.shape[1]))
    for i in range(w.shape[1]):
        actual_tournament[i] = population[int(tournaments[tournament,i])] 
    print actual_tournament
    
    distances = np.zeros(len(actual_tournament))
    for i in range(len(actual_tournament)):
        distances[i] = dist(actual_tournament[i])
    distances = np.sort(distances)
    print distances
    parent1 = distances[0]
    parent2 = distances[1]
    
    return parent1, parent2


    
print mk_parents(w,v,1)
    


[[ 5.  2.  1.  3.  0.  4.]
 [ 0.  2.  5.  1.  4.  3.]
 [ 1.  5.  0.  4.  2.  3.]
 [ 2.  5.  0.  3.  4.  1.]
 [ 0.  1.  3.  2.  5.  4.]
 [ 2.  4.  1.  3.  5.  0.]]
3
[0 1 2 4 3 5]
[[ 0.  1.  2.]
 [ 4.  3.  5.]]
[[ 0.  1.  3.  2.  5.  4.]
 [ 2.  5.  0.  3.  4.  1.]
 [ 2.  4.  1.  3.  5.  0.]]
[ 6467.02  6849.59  7241.25]
(6467.0200000000004, 6849.5900000000001)


/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:13: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/johannes/.local/lib/python2.7/site-packages/ipykernel/__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


[3 0 2 4 1]
[0 1 2 3 4]
